In [90]:
# !pip install arabic-stopwords
import numpy as np
import pandas as pd
import arabicstopwords.arabicstopwords as ast
import re
import string

# Read train data

In [91]:
train_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
train_data.head(10)

,text,category,stance
0,#مريم_رجوي: <LF>حظر خامنئي المجرم شراء #لقاح_كورونا يعد مجزرة متعمدة بحق الشعب الإيراني<LF><LF>نقل موقع مريم رجوي موقف رئيسة الجمهورية المنتخبة للمقاومة الإيرانية من تصريحات خامنئي المجرم حول حظر استيراد لقاح كورونا من الولايات المتحدة و بريطانيا و فرنسا. <LF>#اللقاح_حق_للناس https://t.co/AYXMbXjVKn,info_news,1
1,#الصحة:<LF>•تم إعطاء 259.530 جرعة من لقاح #كورونا<LF>•أهلية الحصول على لقاح كورونا في السعودية وفق الأولوية للمواطن والمقيم ومن خلال تطبيق #صحتي,plan,1
2,#خادم_الحرمين - حفظه الله - يتلقى الجرعة الأولى من لقاح كورونا ( كوفيد - 19 ).<LF>https://t.co/2roLS5gc9u,celebrity,1
3,#الصحه_العالميه: لقاحات #كورونا آمنة ولا خوف منها.. و #كوفاكس يضمن التوزيع العادل<LF>#لقاح_كورونا<LF>للتفاصيل..<LF>https://t.co/jtp8OXyPA4 https://t.co/OGfs5dVmDy,info_news,1
4,"#وزيرة_الصحة ""#هالة_زايد"" تقول إنه يجرى مراجعة خطوط الإنتاج بالشركة القابضة للقاحات لإنتاج لقاح #كورونا في #مصر ثم توفيره إلى الدول الإفريقية https://t.co/TartZnh2Fi",info_news,1
5,2️⃣ وانتهى الفريق من الدراسات قبل السريرية ونشر البحث، وستبدأ التجارب السريرية بعد أخذ الموافقات اللازمة، وقد تم تطوير لقاح سارس-كوف-2، المسبب لكورونا باستخدام تقنية لقاحات الحمض النووي pDNA.,info_news,1
6,عاجل 🔴 <LF>.<LF><LF>.<LF><LF>وزارة الصحة :<LF><LF>كل من حصل على الجرعة الأولى من لقاح كورونا؛ سيأخذ الثانية في موعدها.<LF>.<LF><LF>.<LF>السعودية,plan,1
7,#فيديو | السفير الأميركي لدى السعودية بعد تلقيه لقاح #كوررنا: أؤكد للمجتمع الأميركي هنا أن اللقاح آمن وفعال والتنظيم والكفاءة السعودية اللذان شهدتهما اليوم كانا في غاية الروعة https://t.co/0geJBBZkw8,info_news,1
8,تصريحات وبس الحكومة مع السيسي علي حسب اللقطة! مجاني ولا؟<LF>لقاح كورونا مرة من غير فلوس ومرة بيجمعوا من رجال الاعمال في صندوق تحيا مصر لغير القادرين ،وبعدين وزير المالية بعد ماكان متوفر عندهم فلوس اللقاح دلوقت مفيش معندوش ومستني التبرعات او ضرائب علي المواطن #كورونا #السيسي_عدو_الله https://t.co/exJaBjGoAz,info_news,0
9,"الاتحاد الاوروبي تفاوض لشراء لقاحات الكورونا من الشركات المختلفة ككتلة واحدة، الاسعار اللي حصلو عليها لكل جرعة مطعوم من:<LF><LF>-فايزر/بيونتك: ١٤,٨$ <LF>-استرازانيكا/اكسفورد: ٢,١٩$ <LF>-موديرنا: ١٨ $<LF>-سانوفي: ٩,٣٠ $<LF>-جونسون اند جونسون ٨,٥$",info_news,1


# Pre-Processing the tweets

In [92]:
#getting a stopwords_list
stop_words = ast.stopwords_list()

In [95]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
    
def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text) # no emoji

# a small function to remove stop words
def remove_stop_words(text):
    return ' '.join(word for word in text.split() if word not in stop_words)
    
def processPost(tweet): 

    #Remove <LF> from tweet
    tweet = re.sub('<LF>', ' ', tweet)
    
    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #remove url
    tweet = re.sub('(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})',' ',tweet)
    
    #remove hashtage #
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet = remove_punctuations(tweet)
    
    # remove emoji  (not sure with this step)
    tweet = remove_emoji(tweet)
    
    # normalize the tweet
    # tweet= normalize_arabic(tweet)
    
    # remove repeated letters
    tweet=re.sub(r'(.)\1+', r'\1', tweet)

    #remove stop words
    tweet = remove_stop_words(tweet)
    
    return tweet

In [96]:
train_data["text"] = train_data['text'].apply(lambda x: processPost(x))
train_data.head(10)

,text,category,stance
0,مريمرجوي حظر خامنئي المجرم شراء لقاحكورونا يعد مجزرة متعمدة بحق الشعب الإيراني نقل موقع مريم رجوي موقف رئيسة الجمهورية المنتخبة لمقاومة الإيرانية تصريحات خامنئي المجرم حظر استيراد لقاح كورونا الولايات المتحدة بريطانيا فرنسا القاحقلناس,info_news,1
1,الصحة •تم إعطاء 259530 جرعة لقاح كورونا •أهلية الحصول لقاح كورونا السعودية وفق الأولوية لمواطن والمقيم تطبيق صحتي,plan,1
2,خادمالحرمين حفظه اله يتلقى الجرعة الأولى لقاح كورونا كوفيد 19,celebrity,1
3,الصحهالعالميه لقاحات كورونا آمنة خوف كوفاكس يضمن التوزيع العادل لقاحكورونا لتفاصيل,info_news,1
4,وزيرةالصحة هالةزايد تقول يجرى مراجعة خطوط الإنتاج بالشركة القابضة لقاحات لإنتاج لقاح كورونا مصر توفيره الدول الإفريقية,info_news,1
5,2️⃣ وانتهى الفريق الدراسات السريرية ونشر البحث وستبدأ التجارب السريرية الموافقات الازمة تم تطوير لقاح سارسكوف2 المسب لكورونا باستخدام تقنية لقاحات الحمض النوي pDNA,info_news,1
6,عاجل وزارة الصحة حصل الجرعة الأولى لقاح كورونا سيأخذ الثانية موعدها السعودية,plan,1
7,فيديو السفير الأميركي السعودية تلقيه لقاح كورنا أؤكد لمجتمع الأميركي القاح آمن وفعال والتنظيم والكفاءة السعودية الذان شهدتهما اليوم كانا غاية الروعة,info_news,1
8,تصريحات الحكومة السيسي القطة مجاني لقاح كورونا فلوس بيجمعوا رجال الاعمال صندوق تحيا مصر القادرين وبعدين وزير المالية ماكان متوفر فلوس القاح دلوقت مفيش معندوش ومستني التبرعات او ضرائب المواطن كورونا السيسيعدواله,info_news,0
9,الاتحاد الاوروبي تفاوض لشراء لقاحات الكورونا الشركات المختلفة كتلة واحدة الاسعار الي حصلو جرعة مطعوم فايزربيونتك ١٤٨ استرازانيكاكسفورد ٢١٩ موديرنا ١٨ سانوفي ٩٣٠ جونسون اند جونسون ٨٥,info_news,1


# Feature Extraction 
1. TF-IDF 
2. CBoW
3. Bag of Words

In [105]:
all_words = []
for sentence in train_data["text"]:
    all_words += sentence.split(' ')

vocab = set(all_words)